In [ ]:
import cv2
import numpy as np

# --- Gaussian Blur --- #Smooths the image to *reduce noise* before edge detection.
def gaussian_blur(image, ksize=5):#Defines a function gaussian_blur that takes an image and a kernel size ksize; larger values produce more smoothing.(ksize) 
    return cv2.GaussianBlur(image, (ksize, ksize), 0) #defines the Gaussian window size (odd number required). The last parameter 0 means OpenCV automatically calculates the Gaussian sigma.

# --- Gradient / Canny Functions ---
def compute_gradients(image): #Defines a function to compute gradient magnitudes and directions.
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3) #applies Sobel operator to detect horizontal intensity changes.
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3) #applies Sobel operator to detect vertical intensity changes.
    grad_mag = np.sqrt(grad_x**2 + grad_y**2) #computes gradient magnitude using Euclidean distance formula.
    grad_angle = np.arctan2(grad_y, grad_x)#calculates angle of gradient using arctan2() (range −π to π).
    return grad_mag, grad_angle #Returns both magnitude and angle for later steps.

def non_maximum_suppression(grad_mag, grad_angle): #Thins edges by *suppressing non-maximal pixels* along the gradient direction.
    M, N = grad_mag.shape #Extracts image dimensions M (height) and N (width).
    Z = np.zeros((M, N), dtype=np.float32) #Creates an empty array Z to hold the thinned edges
    angle = grad_angle * 180. / np.pi #Converts gradient angles from radians to degrees.
    angle[angle < 0] += 180 #Ensures angles lie between *0° and 180°*.
    for i in range(1, M-1):
        for j in range(1, N-1): # Compare current pixel with neighbors in gradient direction Loops through the image, avoiding borders (to prevent index errors).
            q = r = 255  # Default comparison values if no direction matches.

            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180): #Horizontal edge → compare left and right.
                q,r = grad_mag[i, j+1], grad_mag[i, j-1] 
            elif 22.5 <= angle[i,j] < 67.5:
                q,r = grad_mag[i+1, j-1], grad_mag[i-1, j+1] #Diagonal edge (down-right/up-left).
            elif 67.5 <= angle[i,j] < 112.5:
                q,r = grad_mag[i+1, j], grad_mag[i-1, j] #Vertical edge → compare above and below.
            elif 112.5 <= angle[i,j] < 157.5:
                q,r = grad_mag[i-1, j-1], grad_mag[i+1, j+1] #Other diagonal.
            Z[i,j] = grad_mag[i,j] if grad_mag[i,j] >= q and grad_mag[i,j] >= r else 0 #Keeps the pixel only if it is *the strongest* in its gradient direction. Else, sets it to 0 (non-edge).
    return Z
   
def double_threshold(image, low, high): 
    strong, weak = 255, 75 #Defines intensity values for strong and weak edges.
    strong_edges = np.zeros_like(image, dtype=np.uint8) #pixels above the high threshold
    weak_edges = np.zeros_like(image, dtype=np.uint8) #pixels between low and high threshold
    strong_edges[image >= high] = strong
    weak_edges[(image >= low) & (image < high)] = weak #Classifies edges by intensity.
    return strong_edges, weak_edges

def edge_tracking(strong_edges, weak_edges):  # Weak edges connected to strong edges become strong
    M, N = strong_edges.shape
    result = np.copy(strong_edges) # Starts the final edge map as a copy of strong edges.
    weak, strong = 75, 255 #Defines pixel values for convenience.
    for i in range(1, M-1):
        for j in range(1, N-1):
            if weak_edges[i,j] == weak: #Loops through all weak edge pixels.
                if np.any(result[i-1:i+2, j-1:j+2] == strong):
                    result[i,j] = strong # Looks at the 3x3 neighborhood.
                                         # Weak becomes strong if connected to a strong edge.
                else: 
                    result[i,j] = 0 #Otherwise, removed
    return result
#Final step of *Canny edge detection*.

In [ ]:
# --- Frequency Domain Filter Masks --- 
def ideal_low_pass(shape, D0): #Pass low frequencies, block high frequencies. (D0)->Cutoff frequency, adjustable via trackbar. 
# shape: the spatial resolution (rows, cols).
# D0: cutoff frequency (radius of the pass region).
    rows, cols = shape #Unpacks the shape into number of rows and columns.
    crow, ccol = rows//2, cols//2 #* Computes the center of the frequency domain (DC component).
    u = np.arange(rows) - crow #vertical frequencies
    v = np.arange(cols) - ccol #horizontal frequencies
    U, V = np.meshgrid(u, v, indexing='ij') #Creates a 2D grid of frequency coordinates:
    D = np.sqrt(U**2 + V**2) #Computes Euclidean distance from the center for each frequency bin.
    mask = (D <= D0).astype(np.float32) #Sets mask = 1 inside the circle of radius D0, 0 outside. sharp cuttoff
    return mask[:, :, np.newaxis] #* Expands mask into 3D to match image channels (H × W × 1).

def ideal_high_pass(shape, D0): # Pass high frequencies, block low frequencies. (D0)->Cutoff frequency, adjustable via trackbar.
    return 1 - ideal_low_pass(shape, D0) #High-pass is simply the complement of low-pass.

def gaussian_low_pass(shape, D0): #Defines a Gaussian low-pass mask.
    rows, cols = shape
    crow, ccol = rows//2, cols//2
    u = np.arange(rows) - crow
    v = np.arange(cols) - ccol
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    mask = np.exp(-(D**2) / (2*D0**2)) #Gaussian attenuation formula:
# Smooth, no sharp edges
# Strong values near center
# Gradually falling

    return mask[:, :, np.newaxis].astype(np.float32) #Output as float32, with channel dimension.

def gaussian_high_pass(shape, D0): #Complement of Gaussian low-pass.
    return 1 - gaussian_low_pass(shape, D0)

def butterworth_low_pass(shape, D0, n=2):
    rows, cols = shape
    crow, ccol = rows//2, cols//2
    u = np.arange(rows) - crow
    v = np.arange(cols) - ccol
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    mask = 1 / (1 + (D/D0)**(2*n)) #* Butterworth formula:

  # Smoother than ideal
  # Adjustable roll-off via n
    return mask[:, :, np.newaxis].astype(np.float32)

def butterworth_high_pass(shape, D0, n=2): #n: filter order controlling steepness.
    return 1 - butterworth_low_pass(shape, D0, n) #Complement of low-pass.

def band_reject(shape, D0, W): #Creates a band-reject filter (also called notch reject). D0: center frequency of rejection band. W: width of rejected band. 
   #Coordinate grid:
    rows, cols = shape
    crow, ccol = rows//2, cols//2
    u = np.arange(rows) - crow
    v = np.arange(cols) - ccol
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    mask = np.ones((rows, cols), dtype=np.float32) #* Start with full pass (all ones).
    mask[(D >= D0 - W/2) & (D <= D0 + W/2)] = 0 #Zero-out the frequencies inside the rejected band.
    return mask[:, :, np.newaxis] #* Add channel axis.

def band_pass(shape, D0, W): #* Inverse of band-reject: keeps only the frequencies inside the band.
    return 1 - band_reject(shape, D0, W)
#Band filters use D0 and W (bandwidth) to selectively pass/reject frequency ranges.

In [ ]:
# --- Trackbar Callback ---
def nothing(x): #* It does nothing because you only need the trackbar values, not an active callback.
    pass

# --- Webcam Setup ---
cap = cv2.VideoCapture(0) #* Opens the default webcam (device index 0).
mode = 'original' #Initializes display mode

cv2.namedWindow('Webcam Filters') #Creates a window where the webcam output will be shown.
cv2.createTrackbar('D0', 'Webcam Filters', 30, 200, nothing) #Trackbar for cutoff frequency *D0* (used by LPF, HPF, Gaussian, Butterworth, etc.).
cv2.createTrackbar('W', 'Webcam Filters', 20, 100, nothing) #Trackbar for *band width* (used in band-pass / band-reject).
cv2.createTrackbar('Canny Low', 'Webcam Filters', 50, 255, nothing)
cv2.createTrackbar('Canny High', 'Webcam Filters', 100, 255, nothing)
# Trackbars for Canny low and high thresholds.
print("Keys: o=Original, c=Canny, 1=ILPF, 2=GLPF, 3=BLPF, 4=IHPF, 5=GHPF, 6=BHPF, 7=Band-Reject, 8=Band-Pass, q=Quit")
#Shows keyboard controls for switching between filter modes
while True:
    ret, frame = cap.read() #Reads one frame from the webcam.
    if not ret:
        break #Breaks loop if camera fails.
# Runs forever until user presses a quit key.

    # --- Mirror the frame ---
    frame = cv2.flip(frame, 1) #Flips horizontally to create a mirror effect.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Converts frame to grayscale for processing.

    D0 = cv2.getTrackbarPos('D0', 'Webcam Filters') #Gets low-pass/high-pass cutoff frequency.
    W = max(cv2.getTrackbarPos('W', 'Webcam Filters'), 1) #* max(..., 1) ensures W is never zero.
    canny_low = cv2.getTrackbarPos('Canny Low', 'Webcam Filters') #* Gets Canny thresholds from UI.
    canny_high = cv2.getTrackbarPos('Canny High', 'Webcam Filters')
   
    if mode == 'original':
        display = frame.copy() #* No processing; show the raw mirrored frame.
    elif mode == 'canny': #* Executes your custom Canny implementation (not OpenCV’s).
        blurred = gaussian_blur(gray) #Smooth image to remove noise
        grad_mag, grad_angle = compute_gradients(blurred) #Compute Sobel gradients.
        nms = non_maximum_suppression(grad_mag, grad_angle) #Thin edges to 1-pixel thickness.
        strong, weak = double_threshold(nms, canny_low, canny_high) #Identify strong and weak edges using thresholds from trackbars
        edges = edge_tracking(strong, weak) #Track weak edges only if connected to strong ones.
        display = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) #Convert edges to BGR to display in color window.
    else: #* Any mode besides 'original' and 'canny' must be a frequency filter.
      #Computes the 2-channel DFT (real + imaginary).
        dft = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft) #Moves low frequencies to center of the spectrum.

        if mode == 'ilpf':
            mask = ideal_low_pass(gray.shape, D0)
        elif mode == 'glpf':
            mask = gaussian_low_pass(gray.shape, D0)
        elif mode == 'blpf':
            mask = butterworth_low_pass(gray.shape, D0)
        elif mode == 'ihpf':
            mask = ideal_high_pass(gray.shape, D0)
        elif mode == 'ghpf':
            mask = gaussian_high_pass(gray.shape, D0)
        elif mode == 'bhpf':
            mask = butterworth_high_pass(gray.shape, D0)
        elif mode == 'breject':
            mask = band_reject(gray.shape, D0, W)
        elif mode == 'bpass':
            mask = band_pass(gray.shape, D0, W)
#Multiplies Fourier spectrum with the selected filter mask.This suppresses or preserves certain frequencies.
        fshift = dft_shift * mask
        f_ishift = np.fft.ifftshift(fshift) #Undo shift to prepare for inverse DFT.
        img_back = cv2.idft(f_ishift) #Converts filtered spectrum back to spatial domain.
        output = cv2.magnitude(img_back[:,:,0], img_back[:,:,1]) #Computes magnitude of complex result
        output = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8) #Normalize values so they display correctly as an image.
        display = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR) #* Convert grayscale result to BGR for window display.

    cv2.putText(display, f"Mode: {mode.upper()}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2) #* Writes text on the displayed frame showing the active mode.

    cv2.imshow('Webcam Filters', display) #Shows the processed frame

    key = cv2.waitKey(1) & 0xFF #Reads key input every frame.
    if key == ord('q'): #quit
        break #Each key sets a different mode
    elif key == ord('o'):
        mode = 'original'
    elif key == ord('c'):
        mode = 'canny'
    elif key == ord('1'):
        mode = 'ilpf'
    elif key == ord('2'):
        mode = 'glpf'
    elif key == ord('3'):
        mode = 'blpf'
    elif key == ord('4'):
        mode = 'ihpf'
    elif key == ord('5'):
        mode = 'ghpf'
    elif key == ord('6'):
        mode = 'bhpf'
    elif key == ord('7'):
        mode = 'breject'
    elif key == ord('8'):
        mode = 'bpass'

cap.release()
cv2.destroyAllWindows()
#* Releases webcam and closes all windows.

Keys: o=Original, c=Canny, 1=ILPF, 2=GLPF, 3=BLPF, 4=IHPF, 5=GHPF, 6=BHPF, 7=Band-Reject, 8=Band-Pass, q=Quit
